In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import *
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import seaborn as sbs
import numpy as np

spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.driver.memory", "4g")
    .config("spark.executer.memory", "8g")
    .getOrCreate()
)

22/09/08 11:43:33 WARN Utils: Your hostname, Luo resolves to a loopback address: 127.0.1.1; using 172.30.203.116 instead (on interface eth0)
22/09/08 11:43:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/08 11:43:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/09/08 11:43:35 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/09/08 11:43:35 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
trans = '../data/curated/transaction cleaned.parquet'
trans_full = '../data/curated/transaction cleaned without remove outlier.parquet/'
merchant = '../data/curated/merchant.csv'

In [3]:
trans_sdf = spark.read.parquet(trans)
trans_full_sdf = spark.read.parquet(trans_full)
merchant_sdf = spark.read.csv(merchant, header = True)

In [4]:
merchant_sdf.count()

4026

In [5]:
trans_sdf.count()

7813531

In [6]:
merged_full_sdf = trans_full_sdf.join(merchant_sdf,trans_full_sdf.merchant_abn ==  merchant_sdf.merchant_abn,"rightouter").drop(merchant_sdf.merchant_abn)

In [7]:
merged_sdf = trans_sdf.join(merchant_sdf,trans_sdf.merchant_abn ==  merchant_sdf.merchant_abn,"rightouter").drop(merchant_sdf.merchant_abn)

In [8]:
merged_sdf = merged_sdf.withColumn('revenue', F.col('dollar_value') * F.col('take_rate') /100)

In [9]:
merged_sdf

user_id,merchant_abn,dollar_value,order_id,order_datetime,is_weekend,DoW,log(dollar),name,tags,revenue_level,take_rate,revenue
16219,10023283211,224.25,a4b4af18-b067-4ac...,2021-11-26,0,6,5.412761500938905,Felis Limited,furniture,e,0.18,0.40365
16516,10023283211,485.9,3bd80282-b6f6-4db...,2021-11-26,0,6,6.186002841411857,Felis Limited,furniture,e,0.18,0.8746199999999998
3047,10023283211,127.1,56fe2a16-85d8-407...,2021-11-26,0,6,4.844974178195408,Felis Limited,furniture,e,0.18,0.22877999999999996
3584,10023283211,890.7,6a8d2f0c-142d-43d...,2021-11-26,0,6,6.792007670437824,Felis Limited,furniture,e,0.18,1.60326
19326,10023283211,245.94,5cc7048f-bfea-4db...,2021-11-26,0,6,5.505087603744301,Felis Limited,furniture,e,0.18,0.442692
4380,10023283211,32.91,4b6ba050-bb75-446...,2021-11-26,0,6,3.4937765629552486,Felis Limited,furniture,e,0.18,0.05923799999999999
4572,10023283211,205.37,4f548c53-d5e4-4ab...,2021-11-26,0,6,5.324813230352002,Felis Limited,furniture,e,0.18,0.369666
20963,10023283211,81.29,2478b44b-2f2a-472...,2021-11-26,0,6,4.398023007758481,Felis Limited,furniture,e,0.18,0.146322
6523,10023283211,90.18,3f5b82d5-ff5f-467...,2021-11-26,0,6,4.501807672992938,Felis Limited,furniture,e,0.18,0.16232400000000002
22450,10023283211,186.24,f1727faa-de41-4c1...,2021-11-26,0,6,5.227036164543073,Felis Limited,furniture,e,0.18,0.33523200000000003


In [10]:
merged_sdf.write.mode('overwrite').parquet("../data/curated/transaction merge merchant.parquet")

In [11]:
missing_full = merged_full_sdf.where(F.col('dollar_value').isNull())

In [12]:
missing_full.count()

7

In [13]:
missing = merged_sdf.where(F.col('dollar_value').isNull())

In [14]:
missing.count()

197

In [15]:
merged_full_sdf.where(F.col('merchant_abn') == 56395390867)

user_id,merchant_abn,dollar_value,order_id,order_datetime,is_weekend,DoW,log(dollar),name,tags,revenue_level,take_rate
